In [34]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

df = pd.read_csv('./sub_id3.csv')

In [21]:
label_encoder = LabelEncoder()
df['ID_encoded'] = label_encoder.fit_transform(df['sub_id'])

In [22]:
# embedding layer 생성

import tensorflow as tf
from tensorflow.keras.layers import Embedding
print(tf.__version__)

n_unique_ids = df['ID_encoded'].nunique()
embedding_dim = min(n_unique_ids // 2, 50)  # 임베딩 차원 설정. 보통은 고유 ID의 수의 절반 혹은 50을 선택

embedding_layer = Embedding(input_dim=n_unique_ids, 
                            output_dim=embedding_dim, 
                            input_length=1, 
                            name='ID_embedding')

2.10.0


In [23]:
df

,sub_id,ID_encoded
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
229,78,38
230,78,38
231,78,38
232,78,38


In [24]:
# 임베딩 레이어를 통과하여 벡터 생성 후, Flatten 적용

from tensorflow.keras.layers import Flatten

# 데이터를 모델에 넣을 수 있는 형태로 변환
input_data = df['ID_encoded'].values.reshape(-1, 1)

# 임베딩 레이어를 통과
embedded_data = embedding_layer(input_data)

# Flatten the output to enable concatenation
sub_id_input = Flatten()(embedded_data)

In [25]:
input_shape = sub_id_input.shape

In [26]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape), 
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(19)
])

# 모델 요약 정보 출력
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 234, 128)          2560      
                                                                 
 dense_7 (Dense)             (None, 234, 64)           8256      
                                                                 
 dense_8 (Dense)             (None, 234, 19)           1235      
                                                                 
Total params: 12,051
Trainable params: 12,051
Non-trainable params: 0
_________________________________________________________________


In [28]:
sub_id_input = tf.expand_dims(sub_id_input, axis = 0)
output = model(sub_id_input)

In [38]:
output = np.squeeze(output, axis = 0)

In [41]:
ids = pd.DataFrame(output, columns = [i for i in range(1, 20)])

In [42]:
ids.to_csv('id_labels.csv')

# Feel One Hot Encoding FC

In [47]:
# 데이터 불러오기
import pandas as pd

df = pd.read_csv('./video_id.csv')
df

,video_id
0,neutral
1,disgust
2,fear
3,sad
4,happy
...,...
229,disgust
230,anger
231,sad
232,happy


In [49]:
# OneHotEncoding
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder()

# Perform one-hot encoding
one_hot_encoded = one_hot_encoder.fit_transform(df['video_id'].values.reshape(-1, 1))

input_data = one_hot_encoded.toarray()
# Convert the result back to a dataframe
#df_encoded = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.categories_[0])

# Convert df_encoded to a suitable input for Keras
input_shape = input_data.shape

#video_id_input = df_encoded.values

In [50]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape), 
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(6)
])

# 모델 요약 정보 출력
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 234, 128)          896       
                                                                 
 dense_14 (Dense)            (None, 234, 64)           8256      
                                                                 
 dense_15 (Dense)            (None, 234, 6)            390       
                                                                 
Total params: 9,542
Trainable params: 9,542
Non-trainable params: 0
_________________________________________________________________


In [52]:
input_data = tf.expand_dims(input_data, axis = 0)
feel_output = model(input_data)

In [53]:
feel_output

<tf.Tensor: shape=(1, 234, 6), dtype=float32, numpy=
array([[[ 0.06554836, -0.07414433, -0.00179375,  0.13654405,
         -0.08512551,  0.07702193],
        [ 0.07658231,  0.02265197,  0.0679667 ,  0.14202967,
         -0.14563766,  0.08540148],
        [ 0.15202275,  0.1284013 , -0.04809507,  0.07894608,
         -0.19606239,  0.03557146],
        ...,
        [ 0.10115917, -0.0490021 , -0.01578688,  0.14458548,
         -0.10818056, -0.0892704 ],
        [-0.0512649 , -0.02255691, -0.08369994,  0.12842439,
         -0.00357763,  0.00918187],
        [ 0.15202275,  0.1284013 , -0.04809507,  0.07894608,
         -0.19606239,  0.03557146]]], dtype=float32)>

In [54]:
feel_output = np.squeeze(feel_output, axis = 0)
feel_output

array([[ 0.06554836, -0.07414433, -0.00179375,  0.13654405, -0.08512551,
         0.07702193],
       [ 0.07658231,  0.02265197,  0.0679667 ,  0.14202967, -0.14563766,
         0.08540148],
       [ 0.15202275,  0.1284013 , -0.04809507,  0.07894608, -0.19606239,
         0.03557146],
       ...,
       [ 0.10115917, -0.0490021 , -0.01578688,  0.14458548, -0.10818056,
        -0.0892704 ],
       [-0.0512649 , -0.02255691, -0.08369994,  0.12842439, -0.00357763,
         0.00918187],
       [ 0.15202275,  0.1284013 , -0.04809507,  0.07894608, -0.19606239,
         0.03557146]], dtype=float32)

In [55]:
feels = pd.DataFrame(feel_output, columns = [i for i in range(1, 7)])

In [56]:
feels

,1,2,3,4,5,6
0,0.065548,-0.074144,-0.001794,0.136544,-0.085126,0.077022
1,0.076582,0.022652,0.067967,0.142030,-0.145638,0.085401
2,0.152023,0.128401,-0.048095,0.078946,-0.196062,0.035571
3,0.101159,-0.049002,-0.015787,0.144585,-0.108181,-0.089270
4,-0.051265,-0.022557,-0.083700,0.128424,-0.003578,0.009182
...,...,...,...,...,...,...
229,0.076582,0.022652,0.067967,0.142030,-0.145638,0.085401
230,0.047362,0.151042,-0.116284,0.095124,0.036967,0.100075
231,0.101159,-0.049002,-0.015787,0.144585,-0.108181,-0.089270
232,-0.051265,-0.022557,-0.083700,0.128424,-0.003578,0.009182


In [57]:
feels.to_csv('feeling_labels.csv')